In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import warnings

In [2]:
# 환경설정
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:,.0f}'.format
if not any([s == 'result' for s in os.listdir('.')]): os.mkdir('result')
BOZ_CD = {
    'A001': '화재',
    'A002': '기술',
    'A003': '종합',
    'A004': '해상',
    'A005': '근재',
    'A006': '배상',
    'A007': '상해',
    'A008': '외국인상해',
    'A009': '농작물재해보상',
    'A010': '기타',
    'A011': '보증',
    'A100': ' ',
}

In [3]:
# 메모
# TODO
# 개별추산액 회수율 반영하기
# 유지비 반영하기

In [13]:
# 데이터 불러오기
# IFRSDB
작업일자 = '20210516'

일반_상품정보 = pd.read_excel(f'data/일반_상품정보_{작업일자}.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_납입예정보험료 = pd.read_excel(f'data/일반_납입예정보험료_{작업일자}.xlsx', dtype={'BSE_YM': str, 'PDC_CD': str, 'PYM_EPCT_YM': str, 'T02_RN_RINSC_CD': str})
일반_원수_미경과보험료 = pd.read_excel(f'data/일반_원수_미경과보험료_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str})
일반_출재_미경과보험료 = pd.read_excel(f'data/일반_출재_미경과보험료_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_개별추산액 = pd.read_excel(f'data/일반_원수_개별추산액_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str})
일반_출재_개별추산액 = pd.read_excel(f'data/일반_출재_개별추산액_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_손해조사비율 = pd.read_excel(f'data/일반_원수_손해조사비율_{작업일자}.xlsx', dtype={'CLG_YM': str, 'RRNR_DAT_DVCD': str, 'PDC_CD': str})
일반_출재_손해조사비율 = pd.read_excel(f'data/일반_출재_손해조사비율_{작업일자}.xlsx', dtype={'CLG_YM': str, 'RRNR_DAT_DVCD': str, 'PDC_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_직전1년경과보험료 = pd.read_excel(f'data/일반_원수_직전1년경과보험료_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str})
일반_출재_직전1년경과보험료 = pd.read_excel(f'data/일반_출재_직전1년경과보험료_{작업일자}.xlsx', dtype={'RRNR_DAT_DVCD': str, 'CLG_YM': str, 'PDC_CD': str, 'T02_RN_RINSC_CD': str})
일반_보험금진전추이 = pd.read_excel(f'data/일반_보험금진전추이_{작업일자}.xlsx', dtype={'BSE_YM': str, 'PDGR_CD': str, 'AY': str})
일반_최종손해율 = pd.read_excel(f'data/일반_최종손해율_{작업일자}.xlsx', dtype={'BSE_YM': 'str', 'RRNR_DAT_DVCD': str, 'PDGR_CD': str})
# 일반_사업비율 = pd.read_excel(f'data/일반_사업비율_{작업일자}.xlsx', dtype={'일반_현행추정부채_집계_익스포져_DATE': 'datetime64[D]', 'RRNR_DVCD': str, 'DMFR_DVCD': str})
# 일반_IBNR율 = pd.read_excel(f'data/일반_IBNR율_{작업일자}.xlsx', dtype={'BASE_DATE': 'datetime64[D]', 'RRNR_DVCD': str, 'DMFR_DVCD': str})
# 일반_보험금진전추이 = pd.read_excel(f'data/일반_보험금진전추이_20200827.xlsx', dtype={'BSE_YM': str, 'PDGR_CD': str, 'AY_YM': str})
일반_사업비율 = pd.read_excel(f'data/일반_사업비율_20200827.xlsx', dtype={'일반_현행추정부채_집계_익스포져_DATE': 'datetime64[D]', 'RRNR_DVCD': str, 'DMFR_DVCD': str})
일반_IBNR율 = pd.read_excel(f'data/일반_IBNR율_20200827.xlsx', dtype={'BASE_DATE': 'datetime64[D]', 'RRNR_DVCD': str, 'DMFR_DVCD': str})
일반_특약정보 = pd.read_excel(f'data/일반_특약정보_{작업일자}.xlsx', dtype={'RNR_TAGR_CD': str, 'CMSN_DVCD': str})
일반_국가구분 = pd.read_excel(f'data/일반_국가구분_{작업일자}.xlsx')

# 기타
금감원_금리시나리오 = pd.read_excel(f'data/금감원_금리시나리오_전체_202012.xlsx', dtype={'MATURITY': int})
재보험자_국내신용등급 = pd.read_excel(f'data/재보험자_국내신용등급_202012.xlsx', dtype={'재보험사코드': str})
선도부도율 = pd.read_excel(f'data/선도부도율_2020.xlsx', dtype={'BASE_YYMM': str, 'YEAR': int})

In [14]:
def 상품군_국가분류_가공(df):
    return df.merge(일반_상품정보_가공, on='PDC_CD', how='left') \
        .assign(PDGR_NM = lambda x: np.where(x.PDC_CD.isin(['10902', '10903']) & (x.NTNL_CTRY_CD == 'KR'), '농작물재해보상', x.PDGR_NM)) \
        .assign(PDGR_NM = lambda x: np.where(x.PDC_CD.isin(['10902', '10903']) & (x.NTNL_CTRY_CD != 'KR'), '기타', x.PDGR_NM)) \
        .assign(PDGR_NM = lambda x: np.where(x.PDC_CD.isin(['10607']), '외국인상해', x.PDGR_NM)) \
        .assign(PDGR_NM = lambda x: np.where(x.PDC_CD.isin(['10011', '10013', '10016', '10017', '10900', '10901']), '종합', x.PDGR_NM)) \
        .assign(PDGR_NM = lambda x: np.where(x.PDGR_NM.isin(['권원']), '기타', x.PDGR_NM)) \
        .assign(NTNL_CTRY_CD = lambda x: x.NTNL_CTRY_CD.fillna('KR')) \
        .assign(DMFR_DVCD = lambda x: np.where(x.NTNL_CTRY_CD == 'KR', '국내', '해외')) \
        .merge(일반_국가구분, left_on='NTNL_CTRY_CD', right_on='CNTR_CD', how='left') \
        .drop(['CNTR_CD_NM', 'NTNL_CTRY_CD'], axis=1)

def 일반_할인요소_계산(일반_보험금진전추이, 금감원_금리시나리오_가공):
    일반_보험금진전추이_가공 = 일반_보험금진전추이.copy()

    할인요소 = []
    for 금리종류 in 금감원_금리시나리오_가공.SCEN_DESC.unique():
        for PDGR_CD in 일반_보험금진전추이_가공['PDGR_CD'].unique():
            일반_보험금진전추이_보종 = 일반_보험금진전추이_가공.query('PDGR_CD == @PDGR_CD').sort_values(by='AY')
            할인요소_보종 = []
            m = len(일반_보험금진전추이_보종) - 1
            for i in range(m):
                n = m - i
                지급보험금 = 0
                for j in range(n):
                    지급보험금 += 일반_보험금진전추이_보종.iloc[m-j, 3+j+1+i] - 일반_보험금진전추이_보종.iloc[m-j, 3+j+i]
                YEAR = i + 1
                평균지급기간 = i + 0.5
                지급보험금 = max(지급보험금, 0)
                할인요소_보종.append([YEAR, 평균지급기간, 지급보험금])
            할인요소_보종 = pd.DataFrame(할인요소_보종, columns=['YEAR', '준비금_평균지급기간', '지급보험금'])
            할인요소_보종['지급보험금_비중'] = 할인요소_보종['지급보험금']/할인요소_보종['지급보험금'].sum()
            할인요소_보종['누적비율'] = 할인요소_보종['지급보험금_비중'].cumsum()
            할인요소_보종['MATURITY'] = (할인요소_보종['준비금_평균지급기간']*12).astype(int)
            할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공.query('SCEN_DESC == @금리종류').drop('SCEN_DESC', axis=1), on='MATURITY', how='left')
            할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
            할인요소_보종.insert(0, 'PDGR_CD', PDGR_CD)
            할인요소_보종['보험료_평균지급기간'] = 할인요소_보종['준비금_평균지급기간'] - 0.2071
            할인요소_보종['조정비율'] = 0
            for i in range(m):
                if i==m-1:
                    할인요소_보종['조정비율'].iloc[i] = 1
                else:
                    할인요소_보종['조정비율'].iloc[i] = 할인요소_보종['누적비율'].iloc[i]+0.2071*(할인요소_보종['누적비율'].iloc[i+1]-할인요소_보종['누적비율'].iloc[i])
            할인요소_보종['1-조정비율'] = 0
            for i in range(m):
                if i==0:
                    할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]
                else:
                    할인요소_보종['1-조정비율'].iloc[i] = 할인요소_보종['조정비율'].iloc[i]-할인요소_보종['조정비율'].iloc[i-1]
            할인요소_보종['MATURITY'] = np.round(할인요소_보종['보험료_평균지급기간']*12).astype(int)
            할인요소_보종 = 할인요소_보종.merge(금감원_금리시나리오_가공.query('SCEN_DESC == @금리종류').drop('SCEN_DESC', axis=1), on='MATURITY', how='left')
            할인요소_보종 = 할인요소_보종.drop(['MATURITY'], axis=1)
            할인요소_보종['SCEN_DESC'] = 금리종류
            할인요소.append(할인요소_보종)

    할인요소 = pd.concat(할인요소) \
        .reset_index(drop=True) \
        .merge(일반_상품군정보, on='PDGR_CD', how='left').drop('PDGR_CD', axis=1)
    준비금부채_할인요소 = 할인요소.groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda df: (df['지급보험금_비중']*df['DISCOUNT_x']).sum()) # 2020.09.25 수정
    보험료부채_할인요소 = 할인요소.groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda df: (df['1-조정비율']*df['DISCOUNT_y']).sum()) # 2020.09.25 수정
    할인요소_집계 = pd.concat([보험료부채_할인요소, 준비금부채_할인요소], axis=1)
    할인요소_집계.columns = ['할인요소_보험료', '할인요소_준비금']
    할인요소_집계 = 할인요소_집계.reset_index()
    
    return 할인요소, 할인요소_집계

def 일반_손실조정_계산(일반_할인요소, 선도부도율):
    # 준비금부채 손실조정
    손실조정_준비금부채 = 일반_할인요소[['SCEN_DESC', 'PDGR_NM', 'YEAR', '지급보험금_비중', 'DISCOUNT_x']] \
        .rename(columns={'DISCOUNT_x': '할인율'}) \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR'], ascending=[True, True, False]) \
        .assign(손실예상액 = lambda x: x.지급보험금_비중*0.5)
    부도시손실액_준비금 = 손실조정_준비금부채 \
        .groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda x: pd.concat([x[['SCEN_DESC', 'PDGR_NM', 'YEAR']], x[['손실예상액']].cumsum()], axis=1)) \
        .rename(columns={'손실예상액': '부도시손실액'}) \
        .reset_index(drop=True)
    손실조정_준비금부채 = 손실조정_준비금부채.merge(부도시손실액_준비금, on=['SCEN_DESC', 'PDGR_NM', 'YEAR'], how='left') \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR']) \
        .merge(선도부도율, on='YEAR', how='left') \
        .assign(준비금_손실조정율 = lambda x: x.FWD_PD*x.부도시손실액*x.할인율) \
        .groupby(['SCEN_DESC', 'PDGR_NM', 'GRADE'])[['준비금_손실조정율']] \
        .sum().reset_index()

    # 보험료부채 손실조정
    손실조정_보험료부채 = 일반_할인요소[['SCEN_DESC', 'PDGR_NM', 'YEAR', '1-조정비율', 'DISCOUNT_y']] \
        .rename(columns={'DISCOUNT_y': '할인율'}) \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR'], ascending=[True, True, False]) \
        .assign(손실예상액 = lambda x: x['1-조정비율']*0.5)
    부도시손실액_보험료 = 손실조정_보험료부채 \
        .groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda x: pd.concat([x[['SCEN_DESC', 'PDGR_NM', 'YEAR']], x[['손실예상액']].cumsum()], axis=1)) \
        .rename(columns={'손실예상액': '부도시손실액'}) \
        .reset_index(drop=True)
    손실조정_보험료부채 = 손실조정_보험료부채.merge(부도시손실액_보험료, on=['SCEN_DESC', 'PDGR_NM', 'YEAR'], how='left') \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR']) \
        .merge(선도부도율, on='YEAR', how='left') \
        .assign(보험료_손실조정율 = lambda x: x.FWD_PD*x.부도시손실액*x.할인율) \
        .groupby(['SCEN_DESC', 'PDGR_NM', 'GRADE'])[['보험료_손실조정율']] \
        .sum().reset_index()

    # 집계
    손실조정 = 손실조정_준비금부채.merge(손실조정_보험료부채, on=['SCEN_DESC', 'PDGR_NM', 'GRADE'], how='outer') \
        .rename(columns={'GRADE': '국내신용등급'})
    손실조정_무위험 = 손실조정.query('국내신용등급 == "AAA"').assign(국내신용등급 = lambda x: '무위험')
#     손실조정_무위험.loc[:, ['보험료_손실조정율', '준비금_손실조정율']] = 0
    손실조정_무등급 = 손실조정.query('국내신용등급 == "B"').assign(국내신용등급 = lambda x: '무등급')
#     손실조정_무등급.loc[:, ['보험료_손실조정율', '준비금_손실조정율']] = 1
    손실조정 = pd.concat([손실조정, 손실조정_무위험, 손실조정_무등급]).reset_index(drop=True)
    
    return 손실조정

# 2020.09.25 추가
def 일반_재보험자산현금흐름비중_계산(일반_할인요소, 선도부도율):
    # 준비금부채 손실조정
    손실조정_준비금부채 = 일반_할인요소[['SCEN_DESC', 'PDGR_NM', 'YEAR', '지급보험금_비중', 'DISCOUNT_x']] \
        .rename(columns={'DISCOUNT_x': '준비금_할인율'}) \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR'], ascending=[True, True, False]) \
        .assign(손실예상액 = lambda x: x.지급보험금_비중*0.5)
    부도시손실액_준비금 = 손실조정_준비금부채 \
        .groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda x: pd.concat([x[['SCEN_DESC', 'PDGR_NM', 'YEAR']], x[['손실예상액']].cumsum()], axis=1)) \
        .rename(columns={'손실예상액': '부도시손실액'}) \
        .reset_index(drop=True)
    손실조정_준비금부채 = 손실조정_준비금부채.merge(부도시손실액_준비금, on=['SCEN_DESC', 'PDGR_NM', 'YEAR'], how='left') \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR']) \
        .merge(선도부도율, on='YEAR', how='left') \
        .assign(준비금_손실조정율 = lambda x: x.FWD_PD*x.부도시손실액*x.준비금_할인율) \
        [['SCEN_DESC', 'PDGR_NM', 'YEAR', '지급보험금_비중', 'GRADE', '준비금_손실조정율', '준비금_할인율']]

    # 보험료부채 손실조정
    손실조정_보험료부채 = 일반_할인요소[['SCEN_DESC', 'PDGR_NM', 'YEAR', '1-조정비율', 'DISCOUNT_y']] \
        .rename(columns={'DISCOUNT_y': '보험료_할인율'}) \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR'], ascending=[True, True, False]) \
        .assign(손실예상액 = lambda x: x['1-조정비율']*0.5)
    부도시손실액_보험료 = 손실조정_보험료부채 \
        .groupby(['SCEN_DESC', 'PDGR_NM']).apply(lambda x: pd.concat([x[['SCEN_DESC', 'PDGR_NM', 'YEAR']], x[['손실예상액']].cumsum()], axis=1)) \
        .rename(columns={'손실예상액': '부도시손실액'}) \
        .reset_index(drop=True)
    손실조정_보험료부채 = 손실조정_보험료부채.merge(부도시손실액_보험료, on=['SCEN_DESC', 'PDGR_NM', 'YEAR'], how='left') \
        .sort_values(by=['SCEN_DESC', 'PDGR_NM', 'YEAR']) \
        .merge(선도부도율, on='YEAR', how='left') \
        .assign(보험료_손실조정율 = lambda x: x.FWD_PD*x.부도시손실액*x.보험료_할인율) \
        [['SCEN_DESC', 'PDGR_NM', 'YEAR', '1-조정비율', 'GRADE', '보험료_손실조정율', '보험료_할인율']]

    # 집계
    현금흐름비중 = 손실조정_준비금부채.merge(손실조정_보험료부채, on=['SCEN_DESC', 'PDGR_NM', 'YEAR', 'GRADE'], how='outer') \
        .rename(columns={'GRADE': '국내신용등급'})
    현금흐름비중_무위험 = 현금흐름비중.query('국내신용등급 == "AAA"').assign(국내신용등급 = lambda x: '무위험')
#     현금흐름비중_무위험.loc[:, ['보험료_손실조정율', '준비금_손실조정율']] = 0
    현금흐름비중_무등급 = 현금흐름비중.query('국내신용등급 == "B"').assign(국내신용등급 = lambda x: '무등급')
#     현금흐름비중_무등급.loc[:, ['보험료_손실조정율', '준비금_손실조정율']] = 1
    현금흐름비중 = pd.concat([현금흐름비중, 현금흐름비중_무위험, 현금흐름비중_무등급]).reset_index(drop=True)

    return 현금흐름비중

In [15]:
# 데이터 가공하기 (1) - 기초정보
# 일반_상품정보
일반_상품군정보 = 일반_상품정보[['PDGR_CD', 'PDGR_NM']].drop_duplicates().assign(PDGR_NM = lambda x: x.PDGR_NM.str.replace('일반/', ''))
일반_상품정보_가공 = 일반_상품정보[['PDC_CD', 'PDGR_NM']].drop_duplicates().assign(PDGR_NM = lambda x: x.PDGR_NM.str.replace('일반/', ''))

# 일반_특약정보
일반_특약정보_가공 = 일반_특약정보 \
    .assign(연동수수료구분 = lambda x: np.where(x.CMSN_DVCD == '1', '연동', '비연동')) \
    .assign(비례비비례구분 = lambda x: np.where((x.RRNR_TTY_RN_TPCD == 'X') | x.TTY_CD_NM.str.contains('KEITI'), '비비례', '비례')) \
    .loc[:, ['RRNR_TTY_CD', 'TTY_CD_NM', '연동수수료구분', '비례비비례구분']]

# 재보험자_국내신용등급
재보험자_국내신용등급_가공 = 재보험자_국내신용등급 \
    .assign(국내신용등급 = lambda x: np.where(x.재보험사코드 == '124106', '무위험', x.국내신용등급))

# 금감원_금리시나리오
금감원_금리시나리오_가공 = 금감원_금리시나리오 \
    .assign(DISCOUNT = lambda x: 1/(1+x.VALUE)**(x.MATURITY/12)) \
    .drop('VALUE', axis=1)

In [16]:
# 데이터 가공하기 (2) - 가정

# 일반_최종손해율
# 수재 가정 원수랑 같음 -> 수재 제거
# 권원/재보험/보증 가정 기타랑 같은 -> 권원/재보험/보증 제거
# 해외 가정 -> 해외원보험 가정으로 사용
# (2021.05.16) 확인
일반_최종손해율_가공 = 일반_최종손해율 \
    .assign(보유구분 = lambda x: x.RRNR_DAT_DVCD.apply(lambda y: {'01': '원수', '02': '수재', '03': '출재'}.get(y, '오류'))) \
    .merge(일반_상품군정보, on='PDGR_CD', how='left') \
    .assign(PDGR_NM = lambda x: x.PDGR_NM.fillna(' ')) \
    .query('PDGR_NM != "권원" and PDGR_NM != "재보험" and PDGR_NM != "보증"') \
    .query('보유구분 != "수재"') \
    .assign(DMFR_DVCD = lambda x: np.where(x.PDGR_NM == "해외원보험", "해외", "국내")) \
    .assign(PDGR_NM = lambda x: x.PDGR_NM.str.replace('해외원보험', ' ')) \
    .pivot_table(index=['DMFR_DVCD', 'PDGR_NM'], columns=['보유구분'], values='FNAL_LSRT', aggfunc=np.sum, fill_value=0) \
    .rename(columns={'원수': '원수_최종손해율', '출재': '출재_최종손해율'}).reset_index()
일반_최종손해율_가공.columns.name = None

# 일반_IBNR율
# (2021.05.16) 확인
일반_IBNR율_가공 = 일반_IBNR율 \
    .assign(보유구분 = lambda x: np.where(x.RRNR_DVCD == '01', '원수', np.where(x.RRNR_DVCD == '03', '출재', '오류'))) \
    .assign(DMFR_DVCD = lambda x: x.KEY.str.slice(0, 1).apply(lambda y: np.where(y == '1', '국내', np.where(y == '2', '해외', '오류')))) \
    .assign(PDGR_CD = lambda x: x.KEY.str.split('_').apply(lambda y: y[-1]).apply(lambda z: np.where(z == '2', ' ', z))) \
    .merge(일반_상품군정보, on='PDGR_CD', how='left') \
    .assign(PDGR_NM = lambda x: x.PDGR_NM.fillna(' ')) \
    .pivot_table(index=['DMFR_DVCD', 'PDGR_NM'], columns=['보유구분'], values='IBNR_RT', aggfunc=np.sum, fill_value=0) \
    .rename(columns={'원수': '원수_IBNR율', '출재': '출재_IBNR율'}).reset_index()
일반_IBNR율_가공.columns.name = None

# 일반_손해조사비율
일반_원수_손해조사비율_가공 = 상품군_국가분류_가공(일반_원수_손해조사비율) \
    .assign(PDGR_NM = lambda x: x.PDGR_NM.apply(lambda y: {'농작물재해보상': '기타', '외국인상해': '상해', '보증': '기타'}.get(y, y))) \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(['DMFR_DVCD', 'PDGR_NM'])[['OGL_PYN_BNF', 'ACCD_IVMT']].sum().astype(float).reset_index() \
    .rename(columns={'OGL_PYN_BNF': '일반_원수_직전3년지급보험금', 'ACCD_IVMT': '일반_원수_직전3년손해조사비'}) \
    .assign(LOSS_ADJ_RATIO = lambda x: np.fmax(x.일반_원수_직전3년손해조사비/x.일반_원수_직전3년지급보험금, 0)) \
    .assign(보유구분 = '원수') \
    .drop(['일반_원수_직전3년지급보험금', '일반_원수_직전3년손해조사비'], axis=1)

일반_출재_손해조사비율_가공 = 상품군_국가분류_가공(일반_출재_손해조사비율) \
    .assign(PDGR_NM = lambda x: x.PDGR_NM.apply(lambda y: {'농작물재해보상': '기타', '외국인상해': '상해', '보증': '기타'}.get(y, y))) \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(['DMFR_DVCD', 'PDGR_NM'])[['WNCR_CNV_PYN_BNF', 'WNCR_CNV_RN_DAG_IVMT']].sum().astype(float).reset_index() \
    .rename(columns={'WNCR_CNV_PYN_BNF': '일반_출재_직전3년지급보험금', 'WNCR_CNV_RN_DAG_IVMT': '일반_출재_직전3년손해조사비'}) \
    .assign(LOSS_ADJ_RATIO = lambda x: np.fmax(x.일반_출재_직전3년손해조사비/x.일반_출재_직전3년지급보험금, 0)) \
    .assign(보유구분 = '출재') \
    .drop(['일반_출재_직전3년지급보험금', '일반_출재_직전3년손해조사비'], axis=1)

일반_손해조사비율_가공 = pd.concat([일반_원수_손해조사비율_가공, 일반_출재_손해조사비율_가공], axis=0) \
    .pivot_table(index=['DMFR_DVCD', 'PDGR_NM'], columns=['보유구분'], values='LOSS_ADJ_RATIO', aggfunc=np.sum, fill_value=0) \
    .rename(columns={'원수': '원수_손해조사비율', '출재': '출재_손해조사비율'}).reset_index()
일반_손해조사비율_가공.columns.name = None

# 일반_사업비율
# 농작물재해보상, 외국인상해, 보증 가정 제거
일반_사업비율_가공 = 일반_사업비율 \
    .assign(PDGR_NM = lambda x: x.BOZ_CD.apply(lambda x: BOZ_CD.get(x, '오류'))) \
    .query('PDGR_NM != "농작물재해보상" and PDGR_NM != "외국인상해" and PDGR_NM != "보증"') \
    .assign(DMFR_DVCD = lambda x: x.DMFR_DVCD.apply(lambda y: {'01': '국내', '02': '해외'}.get(y, '오류'))) \
    .assign(보유구분 = lambda x: x.RRNR_DVCD.apply(lambda y: {'03': '출재', '04': '원수'}.get(y, '오류'))) \
    .pivot_table(index=['DMFR_DVCD', 'PDGR_NM'], columns='보유구분', values='EXP_RT', aggfunc=np.sum, fill_value=0) \
    .assign(원수 = lambda x: np.fmax(x.원수, 0)) \
    .assign(출재 = lambda x: np.fmax(x.출재, 0)) \
    .rename(columns={'원수' : '원수_사업비율', '출재': '출재_사업비율'}).reset_index()
일반_사업비율_가공.columns.name = None

# 일반_할인요소(2020.09.25 해외 할인요소 강제처리 제거)
일반_할인요소, 일반_할인요소_집계 = 일반_할인요소_계산(일반_보험금진전추이, 금감원_금리시나리오_가공)
일반_할인요소_해외 = pd.Series(일반_할인요소_집계['SCEN_DESC'].unique()).to_frame(name='SCEN_DESC')
일반_할인요소_해외.insert(1, 'PDGR_NM', ' ')
일반_할인요소_해외.insert(2, '할인요소_보험료', 1)
일반_할인요소_해외.insert(3, '할인요소_준비금', 1)
일반_할인요소_집계 = pd.concat([일반_할인요소_집계, 일반_할인요소_해외], axis=0).reset_index(drop=True)

# 일반_가정(2020.09.25 해외 삭제)
일반_가정_집계 = 일반_최종손해율_가공 \
    .merge(일반_손해조사비율_가공, on=['DMFR_DVCD', 'PDGR_NM'], how='outer') \
    .merge(일반_IBNR율_가공, on=['DMFR_DVCD', 'PDGR_NM'], how='outer') \
    .merge(일반_사업비율_가공, on=['DMFR_DVCD', 'PDGR_NM'], how='outer') \
    .merge(일반_할인요소_집계, on='PDGR_NM', how='outer')
일반_가정_농작물재해보상 = 일반_가정_집계.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '농작물재해보상')
일반_가정_보증 = 일반_가정_집계.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '보증')
일반_가정_외국인상해 = 일반_가정_집계.query('PDGR_NM == "상해"').assign(PDGR_NM = lambda x: '외국인상해')
일반_가정_집계 = pd.concat([일반_가정_집계, 일반_가정_농작물재해보상, 일반_가정_보증, 일반_가정_외국인상해]).reset_index(drop=True) \
    .eval('원수_손해조사비율_보험료대비 = 원수_손해조사비율*원수_최종손해율') \
    .eval('출재_손해조사비율_보험료대비 = 출재_손해조사비율*출재_최종손해율') \
    .rename(columns = {'DMFR_DVCD': '국내외구분', 'PDGR_NM': '보종', 'SCEN_DESC': '시나리오구분'})
일반_가정_집계_해외 = 일반_가정_집계.copy()
df = 일반_가정_집계_해외.loc[lambda x: x.국내외구분 == "해외"]
tmp = []
for 보종 in 일반_가정_집계_해외.query('보종 != " "')['보종'].unique():
    df.보종 = 보종
    tmp.append(df.copy())
일반_가정_집계 = pd.concat([일반_가정_집계] + tmp).reset_index(drop=True)

# 손실조정
손실조정 = 일반_손실조정_계산(일반_할인요소, 선도부도율)
손실조정_농작물재해보상 = 손실조정.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '농작물재해보상')
손실조정_보증 = 손실조정.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '보증')
손실조정_외국인상해 = 손실조정.query('PDGR_NM == "상해"').assign(PDGR_NM = lambda x: '외국인상해')
손실조정 = pd.concat([손실조정, 손실조정_농작물재해보상, 손실조정_보증, 손실조정_외국인상해]).reset_index(drop=True) \
    .rename(columns = {'PDGR_NM': '보종', 'SCEN_DESC': '시나리오구분'})

# 재보험자산현금흐름비중
재보험자산현금흐름비중 = 일반_재보험자산현금흐름비중_계산(일반_할인요소, 선도부도율)
재보험자산현금흐름비중_농작물재해보상 = 재보험자산현금흐름비중.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '농작물재해보상')
재보험자산현금흐름비중_보증 = 재보험자산현금흐름비중.query('PDGR_NM == "기타"').assign(PDGR_NM = lambda x: '보증')
재보험자산현금흐름비중_외국인상해 = 재보험자산현금흐름비중.query('PDGR_NM == "상해"').assign(PDGR_NM = lambda x: '외국인상해')
재보험자산현금흐름비중 = pd.concat([재보험자산현금흐름비중, 재보험자산현금흐름비중_농작물재해보상, 재보험자산현금흐름비중_보증, 재보험자산현금흐름비중_외국인상해]).reset_index(drop=True) \
    .rename(columns = {'PDGR_NM': '보종', 'SCEN_DESC': '시나리오구분', '1-조정비율': '보험료_비중', '지급보험금_비중': '준비금_비중'}) \
    [['시나리오구분', '보종', '국내신용등급', 'YEAR', '보험료_비중', '보험료_할인율', '보험료_손실조정율', '준비금_비중', '준비금_할인율', '준비금_손실조정율']]

In [17]:
# 데이터 가공하기 (3) - 익스포져

# 일반_원수_미경과보험료
일반_원수_미경과보험료_가공 = 상품군_국가분류_가공(일반_원수_미경과보험료) \
    .assign(PROP_NOT_PROP_DVCD = lambda x: np.where(x.INRI_RN_KND_CD.str.contains('N') & ~x.INRI_RN_KND_CD.isna(), 'NP', np.where(x.RRNR_DAT_DVCD == '01', 'O', 'P'))) \
    .drop('INRI_RN_KND_CD', axis=1)

# 일반_출재_미경과보험료
일반_출재_미경과보험료_가공 = 상품군_국가분류_가공(일반_출재_미경과보험료) \
    .merge(재보험자_국내신용등급_가공, left_on='T02_RN_RINSC_CD', right_on='재보험사코드', how='left') \
    .drop(['재보험사명', '재보험사코드'], axis=1)

# 일반_납입예정보험료
일반_납입예정보험료_가공 = 상품군_국가분류_가공(일반_납입예정보험료) \
    .query('PYM_EPCT_YM > BSE_YM') \
    .merge(재보험자_국내신용등급_가공, left_on='T02_RN_RINSC_CD', right_on='재보험사코드', how='left') \
    .assign(PYM_EPCT_YM = lambda x: pd.to_datetime(x.PYM_EPCT_YM, format='%Y%m')) \
    .assign(BSE_YM = lambda x: pd.to_datetime(x.BSE_YM, format='%Y%m')) \
    .assign(MATURITY = lambda x: x.PYM_EPCT_YM.dt.year*12 + x.PYM_EPCT_YM.dt.month - x.BSE_YM.dt.year*12 - x.BSE_YM.dt.month) \
    .merge(금감원_금리시나리오_가공.query('SCEN_DESC == "BASE"'), on='MATURITY', how='left') \
    .assign(WNCR_CNV_EPCT_PRM_DISC = lambda x: x.WNCR_CNV_EPCT_PRM*x.DISCOUNT) \
    .assign(WNCR_CNV_RN_PRM_DISC = lambda x: x.WNCR_CNV_RN_PRM*x.DISCOUNT)

# 일반_원수_개별추산액
일반_원수_개별추산액_가공 = 상품군_국가분류_가공(일반_원수_개별추산액) \
    .assign(PROP_NOT_PROP_DVCD = lambda x: np.where(x.INRI_RN_KND_CD.str.contains('N') & ~x.INRI_RN_KND_CD.isna(), 'NP', np.where(x.RRNR_DAT_DVCD == '01', 'O', 'P'))) \
    .drop('INRI_RN_KND_CD', axis=1)

# 일반_출재_개별추산액
일반_출재_개별추산액_가공 = 상품군_국가분류_가공(일반_출재_개별추산액) \
    .merge(재보험자_국내신용등급_가공, left_on='T02_RN_RINSC_CD', right_on='재보험사코드', how='left') \
    .drop(['재보험사명', '재보험사코드'], axis=1)

# 일반_원수_직전1년경과보험료
일반_원수_직전1년경과보험료_가공 = 상품군_국가분류_가공(일반_원수_직전1년경과보험료) \
    .assign(PROP_NOT_PROP_DVCD = lambda x: np.where(x.INRI_RN_KND_CD.str.contains('N') & ~x.INRI_RN_KND_CD.isna(), 'NP', np.where(x.RRNR_DAT_DVCD == '01', 'O', 'P'))) \
    .drop('INRI_RN_KND_CD', axis=1)

# 일반_출재_직전1년경과보험료
일반_출재_직전1년경과보험료_가공 = 상품군_국가분류_가공(일반_출재_직전1년경과보험료) \
    .merge(재보험자_국내신용등급_가공, left_on='T02_RN_RINSC_CD', right_on='재보험사코드', how='left') \
    .drop(['재보험사명', '재보험사코드'], axis=1)

In [18]:
# 데이터 집계하기
부채_구분 = ['DMFR_DVCD', 'PDGR_NM']

# 일반_원수_미경과보험료
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['LTPD_URND_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'LTPD_URND_PRM': '일반_원수_미경과보험료'})

# 일반_출재_미경과보험료
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['T02_LTPD_RN_URND_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_LTPD_RN_URND_PRM': '일반_출재_미경과보험료'})

# 일반_원수_납입예정보험료
일반_원수_납입예정보험료_집계 = 일반_납입예정보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['WNCR_CNV_EPCT_PRM_DISC']].sum().astype(float).reset_index() \
    .rename(columns={'WNCR_CNV_EPCT_PRM_DISC': '일반_원수_납입예정보험료'})

# 일반_출재_납입예정보험료
일반_출재_납입예정보험료_집계 = 일반_납입예정보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['WNCR_CNV_RN_PRM_DISC']].sum().astype(float).reset_index() \
    .rename(columns={'WNCR_CNV_RN_PRM_DISC': '일반_출재_납입예정보험료'})

# 일반_원수_개별추산액
일반_원수_개별추산액_집계 = 일반_원수_개별추산액_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['LTPD_OST_AMT']].sum().astype(float).reset_index() \
    .rename(columns={'LTPD_OST_AMT': '일반_원수_개별추산액'})

# 일반_출재_개별추산액
일반_출재_개별추산액_집계 = 일반_출재_개별추산액_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['RN_LTPD_OST_AMT']].sum().astype(float).reset_index() \
    .rename(columns={'RN_LTPD_OST_AMT': '일반_출재_개별추산액'})

# 일반_원수_직전1년경과보험료
일반_원수_직전1년경과보험료_집계 = 일반_원수_직전1년경과보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['ELP_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'ELP_PRM': '일반_원수_직전1년경과보험료'})

# 일반_출재_직전1년경과보험료
일반_출재_직전1년경과보험료_집계 = 일반_출재_직전1년경과보험료_가공 \
    .assign(PDGR_NM = lambda x: np.where(x.DMFR_DVCD == '해외', ' ', x.PDGR_NM)) \
    .groupby(부채_구분)[['T02_RN_ELP_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_RN_ELP_PRM': '일반_출재_직전1년경과보험료'})

# 일반_현행추정부채
일반_현행추정부채_집계 = 일반_원수_미경과보험료_집계 \
    .merge(일반_원수_납입예정보험료_집계, on=부채_구분, how='outer') \
    .merge(일반_원수_개별추산액_집계, on=부채_구분, how='outer') \
    .merge(일반_원수_직전1년경과보험료_집계, on=부채_구분, how='outer') \
    .merge(일반_출재_미경과보험료_집계, on=부채_구분, how='outer') \
    .merge(일반_출재_납입예정보험료_집계, on=부채_구분, how='outer') \
    .merge(일반_출재_개별추산액_집계, on=부채_구분, how='outer') \
    .merge(일반_출재_직전1년경과보험료_집계, on=부채_구분, how='outer') \
    .fillna(value={'일반_원수_미경과보험료': 0, '일반_원수_납입예정보험료': 0, '일반_원수_개별추산액': 0, '일반_원수_직전1년경과보험료': 0}) \
    .fillna(value={'일반_출재_미경과보험료': 0, '일반_출재_납입예정보험료': 0, '일반_출재_개별추산액': 0, '일반_출재_직전1년경과보험료': 0}) \
    .sort_values(by=['DMFR_DVCD', 'PDGR_NM']).rename(columns={'DMFR_DVCD': '국내외구분', 'PDGR_NM': '보종'})

일반_현행추정부채_집계

,국내외구분,보종,일반_원수_미경과보험료,일반_원수_납입예정보험료,일반_원수_개별추산액,일반_원수_직전1년경과보험료,일반_출재_미경과보험료,일반_출재_납입예정보험료,일반_출재_개별추산액,일반_출재_직전1년경과보험료
0,국내,근재,"6,117,369,367","801,690,140","9,198,466,383","16,806,811,403","1,122,556,995","148,950,308","2,119,779,233","2,980,114,181"
1,국내,기술,"31,488,087,780","1,012,621,922","29,934,815,046","32,611,764,250","16,387,806,173","709,418,386","17,570,427,569","18,382,733,489"
2,국내,기타,"87,199,691,334","2,871,822","59,669,955,820","200,409,950,554","47,902,513,309","2,010,272","24,750,172,178","73,846,059,118"
3,국내,농작물재해보상,"1,428,323,657",0,"1,035,744,933","17,492,373,963","112,288,446",0,"1,048,706,061","7,206,404,731"
4,국내,배상,"76,902,451,926","5,393,536,952","50,467,024,337","148,285,278,323","38,787,816,980","1,015,496,373","27,838,374,283","78,310,568,024"
5,국내,보증,"3,681,519",0,"63,123,288","98,738,066",0,0,"59,967,124","92,470,991"
6,국내,상해,"78,522,070,288","15,941,102,924","2,631,796,627","180,857,845,932","28,102,713,482","1,985,172,579","887,684,380","61,970,134,813"
7,국내,외국인상해,"5,677,171,408",0,"25,287,290,701","23,784,002,676",0,0,0,0
8,국내,종합,"122,017,438,052","67,515,997,395","124,180,672,329","196,090,995,409","56,531,957,911","2,732,370,200","66,615,476,611","97,169,594,255"
9,국내,해상,"25,944,357,452","186,719,330,032","93,623,387,811","105,896,990,029","19,283,272,345","13,918,760,645","81,378,289,492","84,213,470,751"


In [19]:
# 일반손해보험 보험료 부채 (일반)
일반_보험료부채 = 일반_현행추정부채_집계.merge(일반_가정_집계.query('시나리오구분 == "BASE"'), on=['국내외구분', '보종'], how='left') \
    .eval('원수_연도말미경과보험료 = 일반_원수_미경과보험료') \
    .eval('원수_보유계약미래보험료 = 일반_원수_납입예정보험료') \
    .eval('원수_Loss = 원수_최종손해율') \
    .eval('원수_LAE = 원수_손해조사비율_보험료대비') \
    .eval('원수_Maint = 원수_사업비율') \
    .eval('출재_연도말미경과보험료 = 일반_출재_미경과보험료') \
    .eval('출재_보유계약미래보험료 = 일반_출재_납입예정보험료') \
    .eval('출재_Loss = 출재_최종손해율') \
    .eval('출재_LAE = 출재_손해조사비율_보험료대비') \
    .eval('출재_Maint = 출재_사업비율') \
    .eval('보유_연도말미경과보험료 = 원수_연도말미경과보험료 - 출재_연도말미경과보험료') \
    .eval('보유_보유계약미래보험료 = 원수_보유계약미래보험료 - 출재_보유계약미래보험료') \
    .eval('보유_Loss = ((원수_연도말미경과보험료+원수_보유계약미래보험료)*원수_Loss - (출재_연도말미경과보험료+출재_보유계약미래보험료)*출재_Loss)/(보유_연도말미경과보험료+보유_보유계약미래보험료)') \
    .eval('보유_LAE = ((원수_연도말미경과보험료+원수_보유계약미래보험료)*원수_LAE - (출재_연도말미경과보험료+출재_보유계약미래보험료)*출재_LAE)/(보유_연도말미경과보험료+보유_보유계약미래보험료)') \
    .eval('보유_Maint = ((원수_연도말미경과보험료+원수_보유계약미래보험료)*원수_Maint - (출재_연도말미경과보험료+출재_보유계약미래보험료)*출재_Maint)/(보유_연도말미경과보험료+보유_보유계약미래보험료)') \
    .eval('할인요소 = 할인요소_보험료') \
    [['국내외구분', '보종',
      '원수_연도말미경과보험료', '원수_보유계약미래보험료', '원수_Loss', '원수_LAE', '원수_Maint', 
      '출재_연도말미경과보험료', '출재_보유계약미래보험료', '출재_Loss', '출재_LAE', '출재_Maint', 
      '보유_연도말미경과보험료', '보유_보유계약미래보험료', '보유_Loss', '보유_LAE', '보유_Maint', 
      '할인요소']]

# 정렬
일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
일반_보험료부채.index = 일반_보험료부채['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
일반_보험료부채 = 일반_보험료부채.sort_index()

# 단위조정(단위: 천원)
단위 = 1000 
일반_보험료부채[['원수_연도말미경과보험료', '원수_보유계약미래보험료',
          '출재_연도말미경과보험료', '출재_보유계약미래보험료',
          '보유_연도말미경과보험료', '보유_보유계약미래보험료']] /= 단위

일반_보험료부채

,국내외구분,보종,원수_연도말미경과보험료,원수_보유계약미래보험료,원수_Loss,원수_LAE,원수_Maint,출재_연도말미경과보험료,출재_보유계약미래보험료,출재_Loss,출재_LAE,출재_Maint,보유_연도말미경과보험료,보유_보유계약미래보험료,보유_Loss,보유_LAE,보유_Maint,할인요소
0,국내,화재,"26,732,515",0,0,0,0,"5,746,312",0,0,0,0,"20,986,203",0,0,0,0,1
1,국내,기술,"31,488,088","1,012,622",0,0,0,"16,387,806","709,418",0,0,0,"15,100,282","303,204",0,-0,0,1
2,국내,종합,"122,017,438","67,515,997",0,0,0,"56,531,958","2,732,370",0,0,0,"65,485,480","64,783,627",0,0,0,1
3,국내,해상,"25,944,357","186,719,330",1,0,0,"19,283,272","13,918,761",1,0,0,"6,661,085","172,800,569",1,0,0,1
4,국내,근재,"6,117,369","801,690",1,0,0,"1,122,557","148,950",1,0,0,"4,994,812","652,740",1,0,0,1
5,국내,배상,"76,902,452","5,393,537",1,0,0,"38,787,817","1,015,496",1,0,0,"38,114,635","4,378,041",1,0,0,1
6,국내,상해,"78,522,070","15,941,103",1,0,0,"28,102,713","1,985,173",1,0,0,"50,419,357","13,955,930",1,0,0,1
7,국내,외국인상해,"5,677,171",0,1,0,0,0,0,1,0,0,"5,677,171",0,1,0,0,1
8,국내,농작물재해보상,"1,428,324",0,1,0,0,"112,288",0,1,0,0,"1,316,035",0,1,0,0,1
9,국내,기타,"87,199,691","2,872",1,0,0,"47,902,513","2,010",1,0,0,"39,297,178",862,1,0,0,1


In [20]:
# 일반손해보험 준비금 부채 (일반)
일반_준비금부채 = 일반_현행추정부채_집계.merge(일반_가정_집계.query('시나리오구분 == "BASE"'), on=['국내외구분', '보종'], how='left') \
    .eval('원수_개별추산 = 일반_원수_개별추산액') \
    .eval('원수_미보고발생손해액 = 일반_원수_미경과보험료*원수_IBNR율') \
    .eval('원수_총량추산 = 원수_개별추산+원수_미보고발생손해액') \
    .eval('원수_장래손해조사비 = (원수_개별추산*0.5+원수_미보고발생손해액*1.0)*원수_손해조사비율') \
    .eval('출재_개별추산 = 일반_출재_개별추산액') \
    .eval('출재_미보고발생손해액 = 일반_출재_미경과보험료*출재_IBNR율') \
    .eval('출재_총량추산 = 출재_개별추산+출재_미보고발생손해액') \
    .eval('출재_장래손해조사비 = (출재_개별추산*0.5+출재_미보고발생손해액*1.0)*출재_손해조사비율') \
    .eval('보유_개별추산 = 원수_개별추산 - 출재_개별추산') \
    .eval('보유_미보고발생손해액 = 원수_미보고발생손해액 - 출재_미보고발생손해액') \
    .eval('보유_총량추산 = 원수_총량추산 - 출재_총량추산') \
    .eval('보유_장래손해조사비 = 원수_장래손해조사비 - 출재_장래손해조사비') \
    .eval('할인요소 = 할인요소_준비금') \
    [['국내외구분', '보종',
      '원수_개별추산', '원수_미보고발생손해액', '원수_총량추산', '원수_장래손해조사비', 
      '출재_개별추산', '출재_미보고발생손해액', '출재_총량추산', '출재_장래손해조사비', 
      '보유_개별추산', '보유_미보고발생손해액', '보유_총량추산', '보유_장래손해조사비', 
      '할인요소']]

# 정렬
일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
일반_준비금부채.index = 일반_준비금부채['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
일반_준비금부채 = 일반_준비금부채.sort_index()

# 단위조정(단위: 천원)
단위 = 1000
일반_준비금부채[['원수_개별추산', '원수_미보고발생손해액', '원수_총량추산', '원수_장래손해조사비', 
      '출재_개별추산', '출재_미보고발생손해액', '출재_총량추산', '출재_장래손해조사비', 
      '보유_개별추산', '보유_미보고발생손해액', '보유_총량추산', '보유_장래손해조사비',]] /= 단위

일반_준비금부채

,국내외구분,보종,원수_개별추산,원수_미보고발생손해액,원수_총량추산,원수_장래손해조사비,출재_개별추산,출재_미보고발생손해액,출재_총량추산,출재_장래손해조사비,보유_개별추산,보유_미보고발생손해액,보유_총량추산,보유_장래손해조사비,할인요소
0,국내,화재,"17,791,879",0,"17,791,879","355,840","5,220,296",0,"5,220,296","100,740","12,571,583",0,"12,571,583","255,100",1
1,국내,기술,"29,934,815","1,449,822","31,384,637","879,873","17,570,428","897,231","18,467,659","1,839,613","12,364,387","552,591","12,916,978","-959,740",1
2,국내,종합,"124,180,672",0,"124,180,672","3,721,308","66,615,477",0,"66,615,477","1,845,681","57,565,196",0,"57,565,196","1,875,627",1
3,국내,해상,"93,623,388","5,008,733","98,632,120","2,599,376","81,378,289","3,658,465","85,036,754","3,149,195","12,245,098","1,350,268","13,595,366","-549,819",1
4,국내,근재,"9,198,466","7,506,529","16,704,995","687,449","2,119,779","1,958,295","4,078,075","117,904","7,078,687","5,548,233","12,626,921","569,545",1
5,국내,배상,"50,467,024","7,537,634","58,004,659","4,949,758","27,838,374","4,118,002","31,956,376","2,959,451","22,628,650","3,419,632","26,048,283","1,990,306",1
6,국내,상해,"2,631,797","15,690,499","18,322,296","602,956","887,684","9,485,722","10,373,406","318,190","1,744,112","6,204,778","7,948,890","284,766",1
7,국내,외국인상해,"25,287,291","1,134,428","26,421,719","488,497",0,0,0,0,"25,287,291","1,134,428","26,421,719","488,497",1
8,국내,농작물재해보상,"1,035,745","108,160","1,143,905","27,072","1,048,706","7,521","1,056,228","6,985","-12,961","100,638","87,677","20,087",1
9,국내,기타,"59,669,956","6,603,188","66,273,143","1,575,728","24,750,172","3,208,685","27,958,857","204,646","34,919,784","3,394,503","38,314,287","1,371,082",1


In [21]:
# 일반손해보험 부채 세부정보 (일반)
일반_부채세부정보 = 일반_현행추정부채_집계.merge(일반_가정_집계.query('시나리오구분 == "BASE"'), on=['국내외구분', '보종'], how='left') \
    .eval('기대미래보험료 = 일반_원수_납입예정보험료*할인요소_보험료') \
    .eval('미래지급보험금 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_최종손해율*할인요소_보험료') \
    .eval('손해조사비 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_손해조사비율_보험료대비*할인요소_보험료') \
    .eval('유지관리비용 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_사업비율*할인요소_보험료') \
    .eval('개별추산_할인전 = 일반_원수_개별추산액') \
    .eval('개별추산 = 개별추산_할인전*할인요소_준비금') \
    .eval('IBNR_할인전 = 일반_원수_미경과보험료*원수_IBNR율') \
    .eval('IBNR = IBNR_할인전*할인요소_준비금') \
    .eval('총량추산 = 개별추산+IBNR') \
    .eval('장래손해조사비 = (개별추산_할인전*0.5+IBNR_할인전*1.0)*원수_손해조사비율*할인요소_준비금') \
    [['국내외구분', '보종', '기대미래보험료', '미래지급보험금', '손해조사비', '유지관리비용', '개별추산', 'IBNR', '총량추산', '장래손해조사비']]

# 정렬
일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
일반_부채세부정보.index = 일반_부채세부정보['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
일반_부채세부정보 = 일반_부채세부정보.sort_index()

# 단위조정(단위: 천원)
단위 = 1000 
일반_부채세부정보[['기대미래보험료', '미래지급보험금', '손해조사비', '유지관리비용', '개별추산', 'IBNR', '총량추산', '장래손해조사비']] /= 단위
일반_부채세부정보

,국내외구분,보종,기대미래보험료,미래지급보험금,손해조사비,유지관리비용,개별추산,IBNR,총량추산,장래손해조사비
0,국내,화재,0,"13,104,435","524,181","1,422,721","17,692,019",0,"17,692,019","353,843"
1,국내,기술,"1,001,753","14,285,903","765,645","785,044","29,510,882","1,429,290","30,940,172","867,412"
2,국내,종합,"67,143,705","76,928,116","4,610,592","6,382,341","123,192,286",0,"123,192,286","3,691,689"
3,국내,해상,"185,342,828","148,183,005","7,433,042","6,198,946","92,679,415","4,958,231","97,637,646","2,573,168"
4,국내,근재,"790,976","6,803,997","386,378","233,588","9,042,058","7,378,890","16,420,948","675,759"
5,국내,배상,"5,329,137","45,975,307","6,944,116","2,923,415","49,695,883","7,422,458","57,118,341","4,874,125"
6,국내,상해,"15,861,798","73,623,676","2,610,302","2,230,352","2,612,889","15,577,775","18,190,664","598,624"
7,국내,외국인상해,0,"4,424,732","156,877","134,043","25,105,620","1,126,278","26,231,898","484,987"
8,국내,농작물재해보상,0,"949,039","41,040","34,138","1,026,631","107,208","1,133,838","26,834"
9,국내,기타,"2,853","57,941,065","2,505,597","2,084,179","59,144,870","6,545,081","65,689,950","1,561,862"


In [22]:
# 금리위험
# 현행추정부채
일반_현행추정부채_금리위험 = 일반_현행추정부채_집계.merge(일반_가정_집계, on=['국내외구분', '보종'], how='left') \
    .eval('기대미래보험료 = 일반_원수_납입예정보험료*할인요소_보험료') \
    .eval('미래지급보험금 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_최종손해율*할인요소_보험료') \
    .eval('손해조사비 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_손해조사비율_보험료대비*할인요소_보험료') \
    .eval('유지관리비용 = (일반_원수_미경과보험료 + 일반_원수_납입예정보험료)*원수_사업비율*할인요소_보험료') \
    .eval('개별추산_할인전 = 일반_원수_개별추산액') \
    .eval('개별추산 = 개별추산_할인전*할인요소_준비금') \
    .eval('IBNR_할인전 = 일반_원수_미경과보험료*원수_IBNR율') \
    .eval('IBNR = IBNR_할인전*할인요소_준비금') \
    .eval('총량추산 = 개별추산+IBNR') \
    .eval('장래손해조사비 = (개별추산_할인전*0.5+IBNR_할인전*1.0)*원수_손해조사비율*할인요소_준비금') \
    .eval('부채평가금액 = -기대미래보험료+미래지급보험금+손해조사비+유지관리비용 + 총량추산+장래손해조사비') \
    .pivot_table(index=['국내외구분', '보종'], columns=['시나리오구분'], values='부채평가금액', aggfunc=np.sum, fill_value=0) \
    [['BASE', 'UP', 'DOWN', 'MEAN-REVERSION', 'TWIST1', 'TWIST2']] \
    .reset_index()

# 정렬
일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
일반_현행추정부채_금리위험.index = 일반_현행추정부채_금리위험['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
일반_현행추정부채_금리위험 = 일반_현행추정부채_금리위험.sort_index()

# 단위조정(단위: 천원)
단위 = 1000
일반_현행추정부채_금리위험[['BASE', 'UP', 'DOWN', 'MEAN-REVERSION', 'TWIST1', 'TWIST2']] /= 단위
일반_현행추정부채_금리위험

시나리오구분,국내외구분,보종,BASE,UP,DOWN,MEAN-REVERSION,TWIST1,TWIST2
0,국내,화재,"33,097,199","33,025,655","33,169,300","33,036,462","33,100,533","33,093,855"
1,국내,기술,"46,642,424","46,340,454","46,950,553","46,492,078","46,619,196","46,665,701"
2,국내,종합,"147,661,320","147,134,823","148,194,178","147,296,737","147,653,261","147,669,159"
3,국내,해상,"76,682,978","76,277,851","77,095,441","76,451,534","76,663,132","76,702,719"
4,국내,근재,"23,729,694","23,540,048","23,924,304","23,648,257","23,713,629","23,745,782"
5,국내,배상,"112,506,167","111,734,639","113,297,410","112,160,946","112,445,869","112,566,594"
6,국내,상해,"81,391,820","81,182,941","81,603,147","81,239,190","81,392,527","81,391,140"
7,국내,외국인상해,"31,432,538","31,331,654","31,534,660","31,360,623","31,432,184","31,432,879"
8,국내,농작물재해보상,"2,184,888","2,176,773","2,193,146","2,179,971","2,184,614","2,185,165"
9,국내,기타,"129,779,800","129,299,701","130,268,282","129,488,835","129,763,560","129,796,151"


In [23]:
# 데이터 집계하기 (신용등급포함)
부채_구분_신용등급포함 = ['DMFR_DVCD', 'PDGR_NM', '국내신용등급']

# 일반_출재_미경과보험료
일반_출재_미경과보험료_집계_신용등급포함 = 일반_출재_미경과보험료_가공 \
    .groupby(부채_구분_신용등급포함)[['T02_LTPD_RN_URND_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_LTPD_RN_URND_PRM': '일반_출재_미경과보험료'})

# 일반_출재_납입예정보험료
일반_출재_납입예정보험료_집계_신용등급포함 = 일반_납입예정보험료_가공 \
    .groupby(부채_구분_신용등급포함)[['WNCR_CNV_RN_PRM_DISC']].sum().astype(float).reset_index() \
    .rename(columns={'WNCR_CNV_RN_PRM_DISC': '일반_출재_납입예정보험료'})

# 일반_출재_개별추산액
일반_출재_개별추산액_집계_신용등급포함 = 일반_출재_개별추산액_가공 \
    .groupby(부채_구분_신용등급포함)[['RN_LTPD_OST_AMT']].sum().astype(float).reset_index() \
    .rename(columns={'RN_LTPD_OST_AMT': '일반_출재_개별추산액'})

# 일반_출재_직전1년경과보험료
일반_출재_직전1년경과보험료_집계_신용등급포함 = 일반_출재_직전1년경과보험료_가공 \
    .groupby(부채_구분_신용등급포함)[['T02_RN_ELP_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_RN_ELP_PRM': '일반_출재_직전1년경과보험료'})

# 일반_현행추정부채_신용등급포함
일반_현행추정부채_집계_신용등급포함 = 일반_출재_미경과보험료_집계_신용등급포함 \
    .merge(일반_출재_납입예정보험료_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .merge(일반_출재_개별추산액_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .merge(일반_출재_직전1년경과보험료_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .fillna(value={'일반_출재_미경과보험료': 0, '일반_출재_납입예정보험료': 0, '일반_출재_개별추산액': 0, '일반_출재_직전1년경과보험료': 0}) \
    .sort_values(by=['DMFR_DVCD', 'PDGR_NM']).rename(columns={'DMFR_DVCD': '국내외구분', 'PDGR_NM': '보종'}) \
    .merge(일반_가정_집계, on=['국내외구분', '보종'], how='left') \
    .merge(손실조정, on=['시나리오구분', '보종', '국내신용등급'], how='left')

# (2020.09.25 추가) 일반_현행추정부채_신용등급 및 시점별 현금흐름비중 포함
일반_현행추정부채_집계_현금흐름비중포함 = 일반_출재_미경과보험료_집계_신용등급포함 \
    .merge(일반_출재_납입예정보험료_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .merge(일반_출재_개별추산액_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .merge(일반_출재_직전1년경과보험료_집계_신용등급포함, on=부채_구분_신용등급포함, how='outer') \
    .fillna(value={'일반_출재_미경과보험료': 0, '일반_출재_납입예정보험료': 0, '일반_출재_개별추산액': 0, '일반_출재_직전1년경과보험료': 0}) \
    .sort_values(by=['DMFR_DVCD', 'PDGR_NM']).rename(columns={'DMFR_DVCD': '국내외구분', 'PDGR_NM': '보종'}) \
    .merge(일반_가정_집계, on=['국내외구분', '보종'], how='left') \
    .merge(재보험자산현금흐름비중, on=['시나리오구분', '보종', '국내신용등급'], how='left')

In [24]:
# 신용위험 (2020.09.25 추가)
일반_현행추정부채_신용위험 = 일반_현행추정부채_집계_현금흐름비중포함.query('시나리오구분 == "BASE"') \
    .eval('보험료부채_할인전 = (일반_출재_납입예정보험료+일반_출재_미경과보험료)*(출재_최종손해율+출재_손해조사비율_보험료대비+출재_사업비율)-일반_출재_납입예정보험료') \
    .eval('준비금부채_할인전 = 일반_출재_개별추산액*(1+0.5*출재_손해조사비율) + 일반_출재_미경과보험료*출재_IBNR율*(1+출재_손해조사비율)') \
    .eval('보험료부채 = 보험료부채_할인전*보험료_비중*보험료_할인율') \
    .eval('준비금부채 = 준비금부채_할인전*준비금_비중*준비금_할인율') \
#     .assign(보종 = lambda x: np.where(x.국내외구분 == "해외", ' ', x.보종)) \
#     .groupby(['YEAR', '국내신용등급'])[['보험료부채', '준비금부채']] \
#     .sum().reset_index()

# # 정렬
# 일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
# 일반_현행추정부채_신용위험.index = 일반_현행추정부채_신용위험['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
# 일반_현행추정부채_신용위험 = 일반_현행추정부채_신용위험.sort_index()

# # 단위조정(단위: 천원)
# 단위 = 1000
# 일반_현행추정부채_신용위험[['보험료부채_보유계약', '보험료부채_미래신계약', '준비금부채', '손실조정']] /= 단위
# (일반_현행추정부채_신용위험['보험료부채']+일반_현행추정부채_신용위험['준비금부채']).sum()/1000
(일반_현행추정부채_신용위험['보험료부채'] + 일반_현행추정부채_신용위험['준비금부채']).sum()/1000

485445080.4689239

In [25]:
# 재보험자산평가
일반_재보험자산평가 = 일반_현행추정부채_집계_신용등급포함.query('시나리오구분 == "BASE"') \
    .eval('보험료부채_할인전 = (일반_출재_납입예정보험료+일반_출재_미경과보험료)*(출재_최종손해율+출재_손해조사비율_보험료대비+출재_사업비율)-일반_출재_납입예정보험료') \
    .eval('준비금부채_할인전 = 일반_출재_개별추산액*(1+0.5*출재_손해조사비율) + 일반_출재_미경과보험료*출재_IBNR율*(1+출재_손해조사비율)') \
    .eval('보험료부채 = 보험료부채_할인전*할인요소_보험료') \
    .eval('준비금부채 = 준비금부채_할인전*할인요소_준비금') \
    .eval('손실조정 = 보험료부채_할인전*보험료_손실조정율 + 준비금부채_할인전*준비금_손실조정율') \
    .assign(보종 = lambda x: np.where(x.국내외구분 == "해외", ' ', x.보종)) \
    .groupby(['국내외구분', '보종'])[['보험료부채', '준비금부채', '손실조정']] \
    .sum().reset_index()

# 정렬
일반_순서 = {'화재': 0, '기술': 1, '종합': 2, '해상': 3, '근재': 4, '배상': 5, '상해': 6, '외국인상해': 7, '농작물재해보상': 8, '기타': 9, '보증': 10, ' ': 11}
일반_재보험자산평가.index = 일반_재보험자산평가['보종'].apply(lambda x: 일반_순서.get(x)).to_numpy()
일반_재보험자산평가 = 일반_재보험자산평가.sort_index()

# 단위조정(단위: 천원)
단위 = 1000
일반_재보험자산평가[['보험료부채', '준비금부채', '손실조정']] /= 단위
# 일반_재보험자산평가['보험료부채'].sum() + 일반_재보험자산평가['준비금부채'].sum() - 일반_재보험자산평가['손실조정'].sum()
# 일반_재보험자산평가['보험료부채'].sum() + 일반_재보험자산평가['준비금부채'].sum()
일반_재보험자산평가

,국내외구분,보종,보험료부채,준비금부채,손실조정
0,국내,화재,"3,232,373","5,291,170","4,435"
1,국내,기술,"8,655,815","20,019,682","26,496"
2,국내,종합,"24,603,292","67,916,257","75,378"
3,국내,해상,"11,932,566","87,296,801","126,075"
4,국내,근재,"1,195,762","4,124,631","5,618"
5,국내,배상,"26,295,174","34,382,310","92,620"
6,국내,상해,"22,932,728","10,614,785","23,738"
8,국내,농작물재해보상,"78,263","1,053,856",811
9,국내,기타,"33,386,485","27,915,668","33,726"
10,국내,보증,0,"119,659",86


In [26]:
# 금리위험
# 재보험자산
일반_재보험자산평가_금리위험 = 일반_현행추정부채_집계_신용등급포함 \
    .eval('보험료부채_보유계약_할인전 = (일반_출재_미경과보험료+일반_출재_납입예정보험료)*(출재_최종손해율+출재_손해조사비율_보험료대비+출재_사업비율)-일반_출재_납입예정보험료') \
    .eval('준비금부채_할인전 = 일반_출재_개별추산액*(1+0.5*출재_손해조사비율) + 일반_출재_미경과보험료*출재_IBNR율*(1+출재_손해조사비율)') \
    .eval('보험료부채_보유계약 = 보험료부채_보유계약_할인전*할인요소_보험료') \
    .eval('준비금부채 = 준비금부채_할인전*할인요소_준비금') \
    .eval('손실조정 = 보험료부채_보유계약_할인전*보험료_손실조정율 + 준비금부채_할인전*준비금_손실조정율') \
    .eval('재보험자산 = 보험료부채_보유계약 + 준비금부채 - 손실조정') \
    .assign(보종 = lambda x: np.where(x.국내외구분 == "해외", ' ', x.보종)) \
    .groupby(['시나리오구분'])[['재보험자산']] \
    .sum().T \
    [['BASE', 'UP', 'DOWN', 'MEAN-REVERSION', 'TWIST1', 'TWIST2']]

# 단위조정(단위: 천원)
단위 = 1000
일반_재보험자산평가_금리위험[['BASE', 'UP', 'DOWN', 'MEAN-REVERSION', 'TWIST1', 'TWIST2']] /= 단위

일반_재보험자산평가_금리위험

시나리오구분,BASE,UP,DOWN,MEAN-REVERSION,TWIST1,TWIST2
재보험자산,"485,794,671","484,029,301","487,592,919","484,787,892","485,711,309","485,878,020"


In [27]:
# 데이터 집계하기 (연동/비연동 및 비례/비비례 포함)
부채_구분_익스포져 = ['KICS_CNTR_CATG_NM', 'PDGR_NM', 'PROP_NOT_PROP_DVCD', 'CMSN_DVCD']

# 일반_원수_개별추산액
일반_원수_개별추산액_집계_익스포져 = 일반_원수_개별추산액_가공 \
    .groupby(부채_구분_익스포져)[['LTPD_OST_AMT']].sum().astype(float).reset_index() \
    .rename(columns={'LTPD_OST_AMT': '일반_원수_개별추산액'})

# 일반_출재_개별추산액
일반_출재_개별추산액_집계_익스포져 = 일반_출재_개별추산액_가공 \
    .groupby(부채_구분_익스포져)[['RN_LTPD_OST_AMT']].sum().astype(float).reset_index() \
    .rename(columns={'RN_LTPD_OST_AMT': '일반_출재_개별추산액'})

# 일반_원수_미경과보험료
일반_원수_미경과보험료_집계_익스포져 = 일반_원수_미경과보험료_가공 \
    .groupby(부채_구분_익스포져)[['LTPD_URND_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'LTPD_URND_PRM': '일반_원수_미경과보험료'})

# 일반_출재_미경과보험료
일반_출재_미경과보험료_집계_익스포져 = 일반_출재_미경과보험료_가공 \
    .groupby(부채_구분_익스포져)[['T02_LTPD_RN_URND_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_LTPD_RN_URND_PRM': '일반_출재_미경과보험료'})

# 일반_원수_직전1년경과보험료
일반_원수_직전1년경과보험료_집계_익스포져 = 일반_원수_직전1년경과보험료_가공 \
    .groupby(부채_구분_익스포져)[['ELP_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'ELP_PRM': '일반_원수_직전1년경과보험료'})

# 일반_출재_직전1년경과보험료
일반_출재_직전1년경과보험료_집계_익스포져 = 일반_출재_직전1년경과보험료_가공 \
    .groupby(부채_구분_익스포져)[['T02_RN_ELP_PRM']].sum().astype(float).reset_index() \
    .rename(columns={'T02_RN_ELP_PRM': '일반_출재_직전1년경과보험료'})

# 일반_현행추정부채_익스포져
일반_현행추정부채_집계_익스포져 = 일반_원수_개별추산액_집계_익스포져 \
    .merge(일반_출재_개별추산액_집계_익스포져, on=부채_구분_익스포져, how='outer') \
    .merge(일반_원수_직전1년경과보험료_집계_익스포져, on=부채_구분_익스포져, how='outer') \
    .merge(일반_출재_직전1년경과보험료_집계_익스포져, on=부채_구분_익스포져, how='outer') \
    .merge(일반_원수_미경과보험료_집계_익스포져, on=부채_구분_익스포져, how='outer') \
    .merge(일반_출재_미경과보험료_집계_익스포져, on=부채_구분_익스포져, how='outer') \
    .assign(DMFR_DVCD = lambda x: np.where(x.KICS_CNTR_CATG_NM == '국내', '국내', '해외')) \
    .fillna(value={'일반_원수_개별추산액': 0, '일반_출재_개별추산액': 0, '일반_원수_직전1년경과보험료': 0, '일반_출재_직전1년경과보험료': 0, '일반_원수_미경과보험료': 0, '일반_출재_미경과보험료': 0}) \
    .rename(columns={'DMFR_DVCD': '국내외구분', 'PDGR_NM': '보종'}) \
    .merge(일반_가정_집계.query('시나리오구분 == "BASE"'), on=['국내외구분', '보종'], how='left') \
    .drop('국내외구분', axis=1)

일반_현행추정부채_집계_익스포져

,KICS_CNTR_CATG_NM,보종,PROP_NOT_PROP_DVCD,CMSN_DVCD,일반_원수_개별추산액,일반_출재_개별추산액,일반_원수_직전1년경과보험료,일반_출재_직전1년경과보험료,일반_원수_미경과보험료,일반_출재_미경과보험료,...,출재_손해조사비율,원수_IBNR율,출재_IBNR율,원수_사업비율,출재_사업비율,시나리오구분,할인요소_보험료,할인요소_준비금,원수_손해조사비율_보험료대비,출재_손해조사비율_보험료대비
0,국내,근재,O,NV,"404,500,421",0,0,0,"184,199,910",0,...,0,1,2,0,0,BASE,1,1,0,0
1,국내,근재,O,V,"6,983,087,720",0,"15,068,702,649",0,"5,593,786,872",0,...,0,1,2,0,0,BASE,1,1,0,0
2,국내,근재,P,NV,"1,810,878,242","820,552,028","1,738,108,754","-39,607,917","339,382,585","475,733",...,0,1,2,0,0,BASE,1,1,0,0
3,국내,기술,O,NV,"27,082,181,269",0,"25,188,187,772",0,"25,662,075,091",0,...,0,0,0,0,0,BASE,1,1,0,0
4,국내,기술,O,V,"2,572,633,777",0,"7,160,516,880",0,"5,510,124,497",0,...,0,0,0,0,0,BASE,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,국내,배상,NP,NV,0,0,0,"4,555,017,344",0,"2,056,802,055",...,0,0,0,0,0,BASE,1,1,0,0
89,미국&캐나다,기타,P,NV,0,0,0,"8,791,806",0,"2,255,672",...,0,0,1,0,0,BASE,1,1,0,0
90,미국&캐나다,보증,P,NV,0,0,0,"187,379,863",0,"90,615,903",...,0,0,1,0,0,BASE,1,1,0,0
91,미국&캐나다,상해,P,NV,0,0,0,"34,495,801",0,"7,753,982",...,0,0,1,0,0,BASE,1,1,0,0


In [28]:
# 일반손해보험리스크 (일반)
# 단위조정(단위: 천원)
# IBNR 수정 필요
단위 = 1000
일반_일반손해보험리스크 = 일반_현행추정부채_집계_익스포져 \
    .eval('원수_보험료_익스포져 = 일반_원수_직전1년경과보험료') \
    .eval('출재_보험료_익스포져 = 일반_출재_직전1년경과보험료') \
    .eval('원수_준비금_익스포져 = (일반_원수_개별추산액*(1+0.5*원수_손해조사비율) + 일반_원수_미경과보험료*원수_IBNR율*(1+원수_손해조사비율))*할인요소_준비금') \
    .eval('출재_준비금_익스포져 = (일반_출재_개별추산액*(1+0.5*출재_손해조사비율) + 일반_출재_미경과보험료*출재_IBNR율*(1+출재_손해조사비율))*할인요소_준비금') \
    .eval('원수_보험료_익스포져 = 원수_보험료_익스포져/@단위') \
    .eval('출재_보험료_익스포져 = 출재_보험료_익스포져/@단위') \
    .eval('원수_준비금_익스포져 = 원수_준비금_익스포져/@단위') \
    .eval('출재_준비금_익스포져 = 출재_준비금_익스포져/@단위') \
    .pivot_table(index=['KICS_CNTR_CATG_NM', '보종'], columns=['CMSN_DVCD', 'PROP_NOT_PROP_DVCD'], values=['원수_보험료_익스포져', '출재_보험료_익스포져', '원수_준비금_익스포져', '출재_준비금_익스포져'], aggfunc=np.sum, fill_value=0) \
    .reset_index()

일반_일반손해보험리스크.head()

KICS_CNTR_CATG_NM       보종 원수_보험료_익스포져              \
CMSN_DVCD                                              NV               
PROP_NOT_PROP_DVCD                                     NP           O   
0                                 국내       근재           0           0   
1                                 국내       기술           0  25,188,188   
2                                 국내       기타           0 196,965,503   
3                                 국내  농작물재해보상      -2,445           0   
4                                 국내       배상           0 112,145,237   

                                            원수_준비금_익스포져                       \
CMSN_DVCD                              V             NV                        
PROP_NOT_PROP_DVCD          P          O  P          NP          O         P   
0                   1,738,109 15,068,703  0           0    643,715 2,263,246   
1                     263,060  7,160,517  0           0 28,641,364   298,539   
2                   3,444,448          0  0           0 66,806,407   445,405   
3                  17,494,819          0  0           0          0 1,160,672   
4                   1,052,168 35,087,874  0           0 46,146,140   826,396   

                    ... 출재_보험료_익스포져                            출재_준비금_익스포져     \
CMSN_DVCD           ...          NV                V                    NV      
PROP_NOT_PROP_DVCD  ...          NP  O          P  O         P          NP  O   
0                   ...      29,025  0    -39,608  0 2,990,697           0  0   
1                   ...     918,516  0 15,091,282  0 2,372,936     872,800  0   
2                   ...           0  0 73,846,059  0         0           0  0   
3                   ...   1,190,910  0  6,015,495  0         0     523,605  0   
4                   ...   4,555,017  0 63,910,918  0 9,844,633     250,310  0   

                                            
CMSN_DVCD                      V            
PROP_NOT_PROP_DVCD          P  O         P  
0                     823,202  0 3,301,429  
1                  18,219,259  0   927,623  
2                  27,915,668  0         0  
3                     530,252  0         0  
4                  29,285,295  0 4,846,706  

[5 rows x 22 columns]

In [29]:
# 데이터 출력하기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/일반_현행추정부채_{now}.xlsx', 'xlsxwriter') as writer:
    일반_보험료부채.to_excel(writer, '일반_보험료부채', index=False)
    일반_준비금부채.to_excel(writer, '일반_준비금부채', index=False)
    일반_부채세부정보.to_excel(writer, '일반_부채세부정보', index=False)
    일반_현행추정부채_금리위험.to_excel(writer, '일반_현행추정부채_금리위험', index=False)
    일반_재보험자산평가.to_excel(writer, '일반_재보험자산평가', index=False)
    일반_재보험자산평가_금리위험.to_excel(writer, '일반_재보험자산평가_금리위험', index=True)
    일반_일반손해보험리스크.to_excel(writer, '일반_일반손해보험리스크', index=True)